## 1. Aufbereitung der Urteilstexte und Vorbereitung des Analyse-Datensatzes

In diesem Abschnitt werden die eingelesenen OpenJur-Urteilstexte weiterverarbeitet und strukturiert. Ziel ist es, relevante Textbestandteile wie den Kopfbereich und den Tenor zu extrahieren, Landgerichtsurteile zu identifizieren und die Daten schließlich in ein geeignetes JSONL-Format für die nachgelagerte automatische Analyse zu überführen.


### 1.1 Import der benötigten Bibliotheken

Zu Beginn werden die für die weitere Verarbeitung erforderlichen Python-Bibliotheken importiert. Diese umfassen Funktionen für Dateizugriffe, reguläre Ausdrücke, Datenverarbeitung mit Pandas sowie den Export der Ergebnisse im JSON-Format.

In [1]:
#Import
import os
import re
import json
import pandas as pd

### 1.2 Einlesen der Urteilstexte und Aufbau des DataFrames

In diesem Schritt werden alle zuvor identifizierten Textdateien zeilenweise eingelesen. Für jede Datei wird eine eindeutige Fall-ID aus dem Dateinamen erzeugt und gemeinsam mit dem vollständigen Text in einem Pandas-DataFrame gespeichert. Der DataFrame bildet die zentrale Datenstruktur für die weitere Analyse.

In [2]:
# (.txt) Dateien einlesen
DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\Lena\Documents\GitHub\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


---

## 2. Textvorverarbeitung und Extraktion zentraler Urteilsbestandteile

In diesem Abschnitt werden die eingelesenen Urteilstexte weiterverarbeitet, um für die nachfolgende Analyse relevante Textbestandteile gezielt zu extrahieren. Hierzu zählen insbesondere ein begrenzter Kopfbereich zur Voranalyse sowie der Tenor als Kern der gerichtlichen Entscheidung. Die strukturierte Aufbereitung dieser Textsegmente bildet die Grundlage für Filter-, Klassifikations- und Extraktionsschritte in den folgenden Abschnitten.

### 2.1 Erzeugung eines Kopfbereichs zur Voranalyse

Da relevante Metadaten wie Gericht, Entscheidungsart und Datum typischerweise am Anfang eines Urteilstextes stehen, wird ein begrenzter Kopfbereich (`head`) aus den ersten Zeichen des Dokuments extrahiert. Dieser verkürzte Textausschnitt dient als effizienter Suchraum für Filter- und Klassifikationsschritte.


In [3]:
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    path = os.path.join(DATA_DIR, fn)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)
print("Gesamt eingelesen:", len(df))


Gesamt eingelesen: 2375


In [4]:
HEAD_CHARS = 8000
df["head"] = df["text"].astype(str).str.slice(0, HEAD_CHARS)

print("Head-Länge (Beispiel):", len(df.loc[0, "head"]))


Head-Länge (Beispiel): 8000


### 2.2 Extraktion des Tenors

Der Tenor enthält die eigentliche gerichtliche Entscheidung und ist daher für die inhaltliche Bewertung besonders relevant. Mithilfe regulärer Ausdrücke wird der Textabschnitt zwischen der Überschrift „Tenor“ und den nachfolgenden Abschnitten (z. B. „Tatbestand“ oder „Gründe“) extrahiert und in einer separaten Spalte gespeichert.

In [5]:
def extract_tenor(text: str) -> str:
    if not isinstance(text, str):
        return ""
    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start:
        return ""
    start = m_start.end()
    m_end = re.search(
        r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b",
        text[start:],
        flags=re.IGNORECASE
    )
    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()

df["tenor"] = df["text"].apply(extract_tenor)

print("Tenor vorhanden:", (df["tenor"].str.len() > 0).sum(), "von", len(df))

Tenor vorhanden: 2362 von 2375


### 2.3 Identifikation von Landgerichtsurteilen (LG)

Im nächsten Schritt werden die Urteile anhand des Kopfbereichs danach gefiltert, ob es sich um Entscheidungen eines Landgerichts handelt. Dazu wird geprüft, ob charakteristische Begriffe wie „Landgericht“ oder die Abkürzung „LG“ im Kopfbereich vorkommen. Auf dieser Grundlage wird eine boolesche Variable erzeugt, die zur Selektion der relevanten Fälle dient.

In [7]:
# Wir suchen nach der Zeile, die mit "Einfach" beginnt, gefolgt von "LG"
# Der Regex r"Einfach\s*\n\s*LG" stellt sicher, dass LG direkt darunter steht
pattern_zitierung_lg = r"Einfach\s*\n\s*LG"

# Wir wenden das auf die Spalte an, die den Kopftext enthält
df["is_landgericht"] = df["head"].str.contains(pattern_zitierung_lg, regex=True, na=False)

# Jetzt erstellen wir den sauberen Dataframe
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"✅ Echte LG-Urteile (über Zitierzeile): {len(df_lg)}")
print("-" * 40)

----------------------------------------
✅ Echte LG-Urteile (über Zitierzeile): 1189
----------------------------------------


In [8]:
import re
import pandas as pd

def split_judgment(text):
    """
    Teilt ein Urteil in Rubrum, Tenor, Tatbestand und Entscheidungsgründe auf.
    """
    segments = {
        "rubrum": "",
        "tenor": "",
        "tatbestand": "",
        "entscheidungsgruende": ""
    }
    
    # Muster für die Abschnittsüberschriften
    # Das Rubrum ist alles vor dem Tenor
    m_tenor = re.search(r"\bTenor\b", text, re.IGNORECASE)
    m_tatbestand = re.search(r"\bTatbestand\b", text, re.IGNORECASE)
    m_gruende = re.search(r"\b(Entscheidungsgründe|Gründe)\b", text, re.IGNORECASE)
    
    if m_tenor:
        segments["rubrum"] = text[:m_tenor.start()].strip()
        
        # Tenor bis Tatbestand
        if m_tatbestand:
            segments["tenor"] = text[m_tenor.end():m_tatbestand.start()].strip()
            
            # Tatbestand bis Gründe
            if m_gruende:
                segments["tatbestand"] = text[m_tatbestand.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tatbestand"] = text[m_tatbestand.end():].strip()
        else:
            # Falls kein Tatbestand gefunden wird, Tenor bis zum Ende oder Gründen
            if m_gruende:
                segments["tenor"] = text[m_tenor.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tenor"] = text[m_tenor.end():].strip()
                
    return segments

# Beispielanwendung auf den Dataframe
df_lg['segments'] = df_lg['text'].apply(split_judgment)

In [9]:
def get_gemini_prompt(segments):
    """
    Erstellt den finalen Prompt basierend auf den Urteilssegmenten.
    """
    prompt = f"""
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
{segments['rubrum']}

TENOR (Ergebnis): 
{segments['tenor']}

TATBESTAND (Sachverhalt): 
{segments['tatbestand']}

ENTSCHEIDUNGSGRÜNDE (Rechtliche Würdigung): 
{segments['entscheidungsgruende']}

### EXTRAKTIONS-AUFGABE:
Extrahiere folgende Variablen (bei Nichtfinden 'null' angeben):

1. **Input-Variablen (Features):**
   - Dieselmotor_Typ: (Beispiel: "EA 189", "EA 288")
   - Art_Abschalteinrichtung: (Beispiel: "Umschaltlogik", "Thermofenster")
   - KBA_Rueckruf: (Boolean: true/false - Beispiel: true)
   - Fahrzeugstatus: ("Neuwagen" oder "Gebrauchtwagen")
   - Fahrzeugmodell_Baureihe: (Beispiel: "VW Golf 2.0 TDI")
   - Update_Status: (Boolean: true/false/null - Beispiel: false)
   - Kilometerstand_Kauf: (Integer - Beispiel: 15200)
   - Kilometerstand_Klageerhebung: (Integer - Beispiel: 45000)
   - Erwartete_Gesamtlaufleistung: (Integer - Beispiel: 250000)
   - Kaufdatum: (Date YYYY-MM-DD - Beispiel: 2014-05-12)
   - Uebergabedatum: (Date YYYY-MM-DD - Beispiel: 2014-05-20)
   - Datum_Klageerhebung: (Date YYYY-MM-DD - Beispiel: 2018-11-03)
   - Nachweis_Aufklaerung: (Boolean: true/false - Beispiel: false)
   - Beklagten_Typ: ("Händler" oder "Hersteller")
   - Datum_Urteil: (Date YYYY-MM-DD - Beispiel: 2019-12-17)
   - Kaufpreis: (Float in EUR - Beispiel: 25900.00)
   - Nacherfuellungsverlangen_Fristsetzung: ("Ja", "Nein", "Entbehrlich")
   - Klageziel: ("Rückabwicklung", "Minderung", "Schadensersatz")
   - Rechtsgrundlage: (Beispiel: "§ 826 BGB", "§ 437 BGB")

2. **Zielvariablen (Labels):**
   - LABEL_Anspruch_Schadensersatz: (Boolean: true/false - Beispiel: true)
   - LABEL_Schadensersatzhoehe_Betrag: (Float in EUR - Beispiel: 18450.50)
   - LABEL_Schadensersatzhoehe_Range: (Beispiel: "15000-20000", "< 5000", "Klage abgewiesen")

### AUSGABEFORMAT:
Antworte NUR mit einem validen JSON-Objekt in einer Liste:
[{{
  "case_id": "...",
  "Dieselmotor_Typ": "...",
  ...
}}]
"""
    return prompt

import json

# Dateiname für den Batch-Input
batch_file_path = "gemini_batch_input_detailed.jsonl"

# Wir nutzen df_lg, das nur die gefilterten Landgerichtsurteile enthält
with open(batch_file_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.iterrows():
        # 1. Urteil in Segmente aufteilen (Funktion aus dem vorherigen Schritt)
        segments = split_judgment(row['text'])
        
        # 2. Den detaillierten Prompt mit den Segmenten generieren
        full_prompt = get_gemini_prompt(segments)
        
        # 3. Payload für die Gemini Batch API strukturieren
        # Jede Zeile muss ein valides JSON-Objekt sein
        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": full_prompt}]
            }]
        }
        
        # 4. Als einzelne Zeile in die Datei schreiben
        f.write(json.dumps(payload) + "\n")

print(f"Erfolgreich erstellt: {batch_file_path} mit {len(df_lg)} Einträgen.")

# Beispielaufruf für den Batch-Job oder Einzeltest
# test_segments = split_judgment(sample_text)
# full_prompt = get_gemini_prompt(test_segments)

Erfolgreich erstellt: gemini_batch_input_detailed.jsonl mit 1189 Einträgen.


In [11]:
# --- TEST-ZELLE: PRÜFUNG DES PROMPTS ---

# 1. Wähle ein Test-Urteil aus deinem gefilterten LG-Datensatz
if not df_lg.empty:
    sample_row = df_lg.iloc[0] # Das erste Urteil in der Liste
    print(f"Teste Prompt für Case: {sample_row['case_id']}...")

    # 2. Schritt: Segmentierung anwenden
    segments = split_judgment(sample_row['text'])

    # 3. Schritt: Prompt generieren
    test_prompt = get_gemini_prompt(segments)

    # 4. Schritt: An Gemini senden (Einzelanfrage)
    # Nutze gemini-2.5-flash für höchste Präzision beim Testen
    model = genai.GenerativeModel("models/gemini-2.5-flash")
    
    try:
        response = model.generate_content(test_prompt)
        print("\n--- ERGEBNIS VON GEMINI ---")
        print(response.text)
    except Exception as e:
        print(f"Fehler bei der API-Abfrage: {e}")
else:
    print("Keine Landgerichtsurteile (LG) im Dataframe gefunden.")

Teste Prompt für Case: 2090187...

--- ERGEBNIS VON GEMINI ---
```json
[
  {
    "case_id": "21 O 658/16",
    "Dieselmotor_Typ": null,
    "Art_Abschalteinrichtung": null,
    "KBA_Rueckruf": true,
    "Fahrzeugstatus": "Neuwagen",
    "Fahrzeugmodell_Baureihe": "VW Passat 2.0 TDI",
    "Update_Status": true,
    "Kilometerstand_Kauf": null,
    "Kilometerstand_Klageerhebung": null,
    "Erwartete_Gesamtlaufleistung": null,
    "Kaufdatum": null,
    "Uebergabedatum": null,
    "Datum_Klageerhebung": null,
    "Nachweis_Aufklaerung": false,
    "Beklagten_Typ": "Händler",
    "Datum_Urteil": "2017-05-23",
    "Kaufpreis": 31234.00,
    "Nacherfuellungsverlangen_Fristsetzung": "Ja",
    "Klageziel": "Rückabwicklung",
    "Rechtsgrundlage": "§ 437 Nr. 1, § 439 Abs. 1 zweite Alternative, § 434 Abs. 1 Satz 2 Nr. 2 BGB",
    "LABEL_Anspruch_Schadensersatz": false,
    "LABEL_Schadensersatzhoehe_Betrag": 0.00,
    "LABEL_Schadensersatzhoehe_Range": "Klage abgewiesen"
  }
]
```


### 2.4 Bereinigung des Kopfbereichs von OLG-Verweisen

Da in vielen Urteilen Verweise auf Oberlandesgerichte (z. B. im Rahmen von Berufungsverfahren) enthalten sind, werden entsprechende Zeilen aus dem Kopfbereich entfernt. Ziel ist es, den Text für die spätere automatische Analyse auf die tatsächlich entscheidungsrelevanten Informationen zu reduzieren und potenzielle Fehlinterpretationen zu vermeiden.

In [12]:
re_olg_line = re.compile(
    r"(?i)^\s*(?:Einfach\s*)?O\s*L\s*G\b|^\s*(?:Einfach\s*)?Oberlandesgericht\b"
)

def remove_olg_lines(block: str) -> str:
    if not isinstance(block, str) or not block:
        return ""
    kept = []
    for line in block.splitlines():
        if re_olg_line.search(line):
            continue
        kept.append(line)
    return "\n".join(kept).strip()

df_lg["head_clean"] = df_lg["head"].apply(remove_olg_lines)

still_olg = df_lg["head_clean"].apply(lambda x: bool(re_olg_line.search(x.splitlines()[0])) if isinstance(x, str) and x else False).sum()

print("🔎 OLG-ZEILEN in head_clean (soll 0 sein):", still_olg)

🔎 OLG-ZEILEN in head_clean (soll 0 sein): 0


### 2.5 Definition des Extraktions- und Analyse-Prompts

Für die spätere automatisierte Auswertung der Urteile wird ein strukturierter Prompt definiert. Dieser enthält detaillierte Anweisungen zur Extraktion von Sachverhaltsmerkmalen, Entscheidungsparametern und Zielvariablen. Der Prompt stellt sicher, dass alle Urteile nach einem einheitlichen Schema analysiert werden.

In [13]:
COMPREHENSIVE_PROMPT = """
Lies den Text genau und analysiere das vorliegende Gerichtsurteil. Extrahiere die folgenden Informationen präzise. Falls eine Information im Text nicht auffindbar ist, gib "null" an.

### Extraktions-Anweisungen:
1. **Input-Variablen (Features):**
    * **Dieselmotor_Typ (String):** Welcher Motortyp (z. B. EA 189, EA 288)?
    * **Art_Abschalteinrichtung (String):** Beschreibung der genannten Abschalteinrichtung.
    * **KBA_Rueckruf (Boolean):** Verpflichtender Rückruf des Kraftfahrt-Bundesamtes? (true/false)
    * **Fahrzeugstatus (String):** "Neuwagen" oder "Gebrauchtwagen"?
    * **Fahrzeugmodell_Baureihe (String):** Bezeichnung des Modells.
    * **Update_Status (Boolean/null):** Software-Update aufgespielt? (true/false/null)
    * **Kilometerstand_Kauf (Integer):** Stand bei Erwerb.
    * **Kilometerstand_Klageerhebung (Integer):** Stand bei Klageeinreichung.
    * **Erwartete_Gesamtlaufleistung (Integer):** Vom Gericht angenommene Gesamtlaufleistung.
    * **Kaufdatum (Date):** Format: YYYY-MM-DD.
    * **Uebergabedatum (Date):** Format: YYYY-MM-DD.
    * **Datum_Klageerhebung (Date):** Format: YYYY-MM-DD.
    * **Nachweis_Aufklaerung (Boolean):** Gab es eine Anlage zum Kaufvertrag über die Software? (true/false)
    * **Beklagten_Typ (String):** "Händler" oder "Hersteller".
    * **Datum_Urteil (Date):** Format: YYYY-MM-DD.
    * **Kaufpreis (Float):** Betrag in Euro (ohne Zinsen).
    * **Nacherfuellungsverlangen_Fristsetzung (String):** "Ja", "Nein", "Entbehrlich".
    * **Klageziel (String):** z. B. "Rückabwicklung", "Minderung", "Schadensersatz".
    * **Rechtsgrundlage (String):** z. B. § 437 BGB oder § 826 BGB.

2. **Zielvariablen (Labeling):**
    * **LABEL_Anspruch_Schadensersatz (Boolean):** Kläger erhält Schadensersatz? (true/false)
    * **LABEL_Schadensersatzhoehe_Betrag (Float):** Zugesprochener Betrag in Euro (ohne Zinsen).
    * **LABEL_Schadensersatzhoehe_Range (String):** "< 5000", "5000-10000", "10001-15000", "> 15000", "Klage abgewiesen".

3. **Kategorisierung "Sonstige":**
    Sollte im Tenor nichts über Abweisung oder Schadensersatz stehen (sondern Streitwertfestsetzung, Ablehnungsgesuche etc.), kategorisiere als "Sonstige". Gib im Feld "Urteil_Anmerkung" die Begründung an.

### Ausgabeformat:
Antworte AUSSCHLIESSLICH als JSON-Liste mit einem Eintrag. Ignoriere Streitwerte und Zinsen.
""".strip()

### 2.6 Export der gefilterten LG-Urteile im JSONL-Format

Abschließend werden ausschließlich die gefilterten Landgerichtsurteile in eine JSONL-Datei exportiert. Jedes Urteil wird dabei als einzelner Eintrag mit Fall-ID, bereinigtem Kopfbereich, Tenor und Analyse-Prompt gespeichert. Dieses Format eignet sich insbesondere für die Batch-Verarbeitung mit großen Sprachmodellen.

In [14]:
output_path = "gemini_batch_input_NUR_LG.jsonl"

export_count = 0
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.iterrows():
        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{
                    "text": f"Kopf: {row['head_clean']}\nTenor: {row['tenor']}\n\n{COMPREHENSIVE_PROMPT}"
                }]
            }]
        }
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")
        export_count += 1

print("✅ JSONL geschrieben:", output_path, "| Zeilen:", export_count)

✅ JSONL geschrieben: gemini_batch_input_NUR_LG.jsonl | Zeilen: 1189


---

## 3. Validierung der Batch-Verarbeitung anhand eines Einzelfalls

Bevor die vollständige Batch-Verarbeitung aller Landgerichtsurteile durchgeführt wird, erfolgt ein Probelauf anhand eines einzelnen Dokuments. Dazu wird aus der zuvor erzeugten JSONL-Datei eine reduzierte Testdatei erstellt, die ausschließlich den ersten Eintrag enthält.

Dieser Einzelfall dient der inhaltlichen Validierung des Prompts und der erwarteten Modellantwort.
Da die Gemini-Batch-API im kostenlosen Zugang nicht zuverlässig nutzbar ist, erfolgt die tatsächliche Verarbeitung im nächsten Schritt über eine sequentielle Simulation.

### 3.1 Erstellung einer Test-JSONL-Datei für den Probelauf

Zur technischen und inhaltlichen Validierung der Batch-Verarbeitung wird zunächst eine reduzierte JSONL-Datei erstellt, die ausschließlich einen einzelnen Fall enthält. Dieser Probelauf ermöglicht eine gezielte Überprüfung des Prompts und der Modellantwort, bevor die Analyse auf den vollständigen Datensatz ausgeweitet wird.

In [15]:
# Pfade
input_jsonl = "gemini_batch_input_NUR_LG.jsonl"
test_jsonl  = "gemini_batch_input_TEST_1.jsonl"

# Erste Zeile aus der großen JSONL kopieren
with open(input_jsonl, "r", encoding="utf-8") as fin:
    first_line = fin.readline()

with open(test_jsonl, "w", encoding="utf-8") as fout:
    fout.write(first_line)

print("✅ Test-JSONL mit einem Dokument erstellt:", test_jsonl)


✅ Test-JSONL mit einem Dokument erstellt: gemini_batch_input_TEST_1.jsonl


### 3.2 Durchführung eines simulierten Batch-Probelaufs (API-freie Validierung)

Da für die Projektarbeit kein kostenpflichtiger API-Plan zur Verfügung steht, wurde anstelle eines echten Batch-Jobs eine sequenzielle Simulation der Batch-Verarbeitung durchgeführt.

Hierbei wird jedes Dokument der Test-JSONL-Datei nacheinander über die Gemini-API verarbeitet. Dieses Vorgehen bildet die Logik einer Batch-Verarbeitung nach, ohne gleichzeitig mehrere Anfragen zu starten oder API-Limits zu überschreiten.

Ziel dieses Schritts ist die technische Validierung des Prompts, der Eingabestruktur sowie des erwarteten JSON-Ausgabeformats, bevor die Analyse auf den vollständigen Datensatz ausgeweitet wird.



### 3.3 Sichtprüfung der Modellantwort

Die vom Modell erzeugte Ausgabe des simulierten Einzelfalls wird anschließend manuell überprüft. Dabei wird kontrolliert, ob die extrahierten Merkmale vollständig, konsistent und in dem erwarteten JSON-Format vorliegen. Auf Grundlage dieser Sichtprüfung wird entschieden, ob der Prompt oder die Vorverarbeitung angepasst werden müssen.

## 4. Simulierte automatisierte Urteilsanalyse (Pilot-Datensatz)

Ziel des Abschnitts:
Ziel dieses Abschnitts ist die Durchführung einer simulierten automatisierten Urteilsanalyse auf Basis eines Pilot-Datensatzes.

Aufgrund von API-Limitierungen konnte die Analyse nicht auf den vollständigen Datensatz ausgeweitet werden. Stattdessen wurde ein kontrollierter Teil der Landgerichtsurteile automatisiert ausgewertet, um die prinzipielle Funktionsfähigkeit der Pipeline, die Plausibilität der extrahierten Zielvariablen sowie die technische Umsetzbarkeit der Methode nachzuweisen.

Die nachfolgenden Schritte zeigen exemplarisch, wie die vollständige Analyse bei Verfügbarkeit ausreichender API-Ressourcen skaliert werden könnte.


> Hinweis: Der folgende Code zeigt den prinzipiellen Ablauf eines echten Batch-Jobs, 
> wurde im Rahmen dieser Arbeit jedoch nicht ausgeführt.

In [ ]:
import os 
import time, random
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def create_batch_with_backoff(*, model, src, display_name, retries=8):
    delay = 2.0
    for attempt in range(1, retries + 1):
        try:
            return client.batches.create(
                model=model,
                src=src,
                config={"display_name": display_name},
            )
        except Exception as e:
            msg = str(e).upper()
            if "429" in msg or "RESOURCE_EXHAUSTED" in msg:
                sleep_s = delay + random.uniform(0, 0.5 * delay)
                print(f"[{attempt}/{retries}] 429 -> sleep {sleep_s:.1f}s")
                time.sleep(sleep_s)
                delay = min(delay * 2, 60)
                continue
            raise

# Upload der Test-JSONL-Datei
uploaded = client.files.upload(
    file="gemini_batch_input_TEST_1.jsonl",
    config={"display_name": "diesel-lg-test-1", "mime_type": "application/jsonl"},
)
print("Upload erfolgreich", uploaded.name)

# Start des Batch-Jobs
job = create_batch_with_backoff(
    model="gemini-2.5-flash",
    src=uploaded.name,
    display_name="diesel-lg-test-1",
)
print("Batch:", job.name)


Upload erfolgreich files/sjnsdwmm49mm
[1/8] 429 -> sleep 2.0s
[2/8] 429 -> sleep 5.2s
[3/8] 429 -> sleep 8.3s
[4/8] 429 -> sleep 20.8s
[5/8] 429 -> sleep 42.8s
[6/8] 429 -> sleep 77.0s
[7/8] 429 -> sleep 64.6s
[8/8] 429 -> sleep 81.2s


AttributeError: 'NoneType' object has no attribute 'name'

**Hinweis zur technischen Umsetzung:**

Aufgrund von API-Limitierungen (Free-Tier, Rate Limits) konnte die vollständige 
Batch-Verarbeitung aller Urteile nicht durchgeführt werden. 
Stattdessen wurde eine sequenzielle Simulation der Batch-API implementiert, 
um die prinzipielle Funktionsfähigkeit der Pipeline, die Plausibilität der 
extrahierten Zielvariablen sowie die technische Umsetzbarkeit des Ansatzes zu demonstrieren.


In [19]:
import json, time

INPUT_JSONL = "gemini_batch_input_NUR_LG.jsonl"
OUTPUT_JSONL = "simulated_batch_output_ALL.jsonl"

def get_id(item, fallback):
    return item.get("custom_id") or item.get("id") or f"line-{fallback}"

def extract_prompt(item: dict):
    return item.get("contents") or item.get("prompt") or str(item)

with open(INPUT_JSONL, "r", encoding="utf-8") as fin, open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
    for idx, line in enumerate(fin, start=1):
        item = json.loads(line)
        cid = get_id(item, idx)
        prompt = extract_prompt(item)

        try:
            resp = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=prompt
            )
            fout.write(json.dumps({
                "custom_id": cid,
                "status": "OK",
                "text": getattr(resp, "text", None),
            }, ensure_ascii=False) + "\n")
        except Exception as e:
            fout.write(json.dumps({
                "custom_id": cid,
                "status": "ERROR",
                "error": str(e),
            }, ensure_ascii=False) + "\n")

        time.sleep(0.2)

print("Fertig:", OUTPUT_JSONL)


KeyboardInterrupt: 

**Technischer Hinweis (nicht Teil der Analyse):**

Die nachfolgende Fortsetzungslogik wurde ausschließlich zur
technischen Absicherung der Simulation eingesetzt, um bei
Rate-Limit-Fehlern (HTTP 429) bereits erfolgreich verarbeitete
Dokumente nicht erneut anfragen zu müssen.
Für die wissenschaftliche Analyse wird dieser Mechanismus
nicht benötigt und nicht berücksichtigt.


In [23]:
import json, time, os

INPUT_JSONL = "gemini_batch_input_NUR_LG.jsonl"
OUTPUT_JSONL = "simulated_batch_output_ALL.jsonl"

# 1) IDs laden, die schon OK sind
done_ids = set()
with open(OUTPUT_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        try:
            obj = json.loads(line)
            if obj.get("status") == "OK":
                done_ids.add(obj.get("custom_id"))
        except Exception:
            pass

print("Schon OK verarbeitet:", len(done_ids))

# 2) Weiterlaufen: append statt write
with open(INPUT_JSONL, "r", encoding="utf-8") as fin, open(OUTPUT_JSONL, "a", encoding="utf-8") as fout:
    for idx, line in enumerate(fin, start=1):
        item = json.loads(line)
        cid = item.get("custom_id")

        # alles überspringen, was schon OK ist
        if cid in done_ids:
            continue

        try:
            resp = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=item["contents"]
            )
            fout.write(json.dumps({
                "custom_id": cid,
                "status": "OK",
                "text": getattr(resp, "text", None),
            }, ensure_ascii=False) + "\n")
            fout.flush()
        except Exception as e:
            fout.write(json.dumps({
                "custom_id": cid,
                "status": "ERROR",
                "error": str(e),
            }, ensure_ascii=False) + "\n")
            fout.flush()
            print("Stop wegen Fehler (wahrscheinlich 429):", e)
            break

        time.sleep(1.0)

print("Weiterlauf beendet.")


Schon OK verarbeitet: 19
Stop wegen Fehler (wahrscheinlich 429): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 35.914916024s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': 

**Selektion valider Modellantworten:**

Da im Rahmen der Simulation einzelne Modellanfragen aufgrund
technischer Limitierungen (z. B. API-Rate-Limits) fehlschlagen können,
werden im folgenden Schritt ausschließlich erfolgreich erzeugte
Modellantworten (`status = "OK"`) extrahiert.

Diese valide Teilmenge dient als Grundlage für die weitere
Datenaufbereitung und Auswertung.



In [ ]:
import json

INPUT = "simulated_batch_output_ALL.jsonl"
OUTPUT = "simulated_batch_output_OK_ONLY.jsonl"

ok_count = 0

with open(INPUT, "r", encoding="utf-8") as fin, open(OUTPUT, "w", encoding="utf-8") as fout:
    for line in fin:
        obj = json.loads(line)
        if obj.get("status") == "OK":
            fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
            ok_count += 1

print("OK-Zeilen geschrieben:", ok_count)
print("Neue Datei:", OUTPUT)


OK-Zeilen geschrieben: 19
Neue Datei: simulated_batch_output_OK_ONLY.jsonl


In diesem Schritt lesen wir die erfolgreichen Modellantworten aus und holen daraus die beiden Zielvariablen (Anspruch ja/nein und Betrag). Diese speichern wir tabellarisch, damit wir sie später mit den Urteilstexten zusammenführen und auswerten können.

In [30]:
import json, re
import pandas as pd

INPUT = "simulated_batch_output_OK_ONLY.jsonl"

def parse_json_from_llm_text(text: str):
    text = re.sub(r"^```json\s*|\s*```$", "", text.strip(), flags=re.MULTILINE)
    data = json.loads(text)
    if isinstance(data, list) and len(data) > 0:
        return data[0]
    return data

rows = []

with open(INPUT, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        case_id = obj["custom_id"].replace("case_", "")
        extracted = parse_json_from_llm_text(obj["text"])

        extracted["case_id"] = case_id
        rows.append(extracted)


df_labels = pd.DataFrame(rows)
df_labels


,Dieselmotor_Typ,Art_Abschalteinrichtung,KBA_Rueckruf,Fahrzeugstatus,Fahrzeugmodell_Baureihe,Update_Status,Kilometerstand_Kauf,Kilometerstand_Klageerhebung,Erwartete_Gesamtlaufleistung,Kaufdatum,...,Datum_Urteil,Kaufpreis,Nacherfuellungsverlangen_Fristsetzung,Klageziel,Rechtsgrundlage,LABEL_Anspruch_Schadensersatz,LABEL_Schadensersatzhoehe_Betrag,LABEL_Schadensersatzhoehe_Range,case_id,Urteil_Anmerkung
0,NaN,NaN,None,Neuwagen,"VW Passat 2,0 l TDI",None,NaN,NaN,NaN,NaN,...,2017-05-23,31234.00,Nein,Nachlieferung,"§ 437 Nr. 1, § 439 Abs. 1 zweite Alternative, ...",False,0.00,Klage abgewiesen,2090187,NaN
1,EA 189,"Motorensteuerungsgerätesoftware, die erkennt, ...",True,Neuwagen,"1,6l TDI",None,NaN,NaN,NaN,NaN,...,2018-05-18,28486.76,Entbehrlich,Rückabwicklung,§ 826 BGB,True,18412.37,> 15000,2112111,NaN
2,EA 189,"Software, die zwischen zwei Betriebsmodi (Modu...",True,Gebrauchtwagen,VW Touran,None,10200.0,NaN,NaN,2013-11-15,...,2017-06-30,20900.00,Ja,Rückabwicklung,NaN,True,15710.80,> 15000,2112115,NaN
3,EA 189,"Zwei Betriebsmodi (0 und 1), wobei im Modus 1 ...",True,Neuwagen,VW Touran Comfortline 1.6 l TDI,False,NaN,NaN,NaN,2014-10-02,...,2017-08-17,31377.75,Nein,Rückabwicklung,NaN,True,1358.86,< 5000,2112117,NaN
4,EA189,"Installierte Software erkennt, wann sich das F...",True,Neuwagen,VW Touran Comfortline,None,NaN,28962.0,NaN,NaN,...,2017-09-08,27000.01,Entbehrlich,Rückabwicklung,"§§ 346 Abs. 1, 437 Nr. 2, 323 Abs. 1, Abs. 2 N...",False,NaN,NaN,2112118,NaN
5,EA 189,"Software erkennt, ob sich das Kfz auf einem te...",True,Gebrauchtwagen,VW Touran,None,NaN,NaN,NaN,2015-06-23,...,2017-07-12,31999.00,Entbehrlich,Rückabwicklung und Schadensersatz,NaN,True,468.74,< 5000,2112119,NaN
6,EA 189,"Software, die zwischen Prüfstandbetrieb (Modus...",True,Neuwagen,VW Touran,False,NaN,NaN,NaN,2013-04-19,...,2017-06-07,28068.00,Ja,Rückabwicklung,NaN,True,9872.20,5000-10000,2112121,NaN
7,EA 189,Zwei Betriebsmodi der Motorsteuerung für Abgas...,True,Gebrauchtwagen,VW Beetle Cabrio,False,NaN,NaN,NaN,2015-06-01,...,2018-08-09,27390.00,Entbehrlich,Schadensersatz,§ 826 BGB,True,1358.86,< 5000,2112123,NaN
8,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,...,2017-04-25,NaN,NaN,"Rückabwicklung, Schadensersatz","§ 823 Abs. 2 BGB i.V.m. § 263 StGB, § 826 BGB",False,NaN,Sonstige,2124977,Das Gericht lehnt die Bestimmung des zuständig...
9,NaN,Abgasrückführungsraten wurden unterhalb einer ...,True,Neuwagen,Porsche Macan S Diesel,False,0.0,NaN,NaN,NaN,...,2018-10-30,74500.00,Entbehrlich,Schadensersatz,NaN,True,0.00,NaN,2126821,NaN


### 4.1 Bereitstellung der Analyse-Eingabedaten

Die in Abschnitt 2 aufbereiteten und gefilterten Urteilstexte liegen bereits in Form einer strukturierten JSONL-Datei vor.
Diese Datei dient als direkter Input für die nachgelagerte automatisierte Analyse und wird im Folgenden in das Batch-Verarbeitungssystem eingelesen.

(gemini_batch_input_NUR_LG.jsonl)

In [ ]:
# Hier Code einfügen

### 4.2 Übergabe des Analyse-Datensatzes an die Batch-API

Nach der in Abschnitt 2 beschriebenen Aufbereitung der Urteilstexte liegt der vollständige Analyse-Datensatz in Form einer strukturierten JSONL-Datei vor. Diese Datei dient in diesem Schritt als Eingabe für die automatisierte Verarbeitung durch ein großes Sprachmodell.

Die JSONL-Datei wird zunächst in das Batch-System hochgeladen. Anschließend wird ein Batch-Verarbeitungsjob gestartet, der die hochgeladene Datei als Eingabequelle verwendet. Für jedes enthaltene Dokument erzeugt das Modell eine strukturierte Antwort gemäß den im Prompt definierten Extraktionsvorgaben.

Als Ergebnis des Batch-Jobs stellt die API eine Ausgabedatei bereit, die die Modellantworten zu allen verarbeiteten Urteilen enthält. Diese Ausgabedatei liegt ebenfalls im JSONL-Format vor und bildet die Grundlage für die weitere Aufbereitung und Auswertung der Ergebnisse.


In [ ]:
# Hier Code einfügen

### 4.3 Aufbereitung der Batch-Ergebnisse und Erstellung des Analyse-Datensatzes

Die im vorherigen Schritt erzeugte Ausgabedatei des Batch-Jobs liegt zunächst als Rohdaten im JSONL-Format vor. Jede Zeile dieser Datei enthält die strukturierte Modellantwort zu einem einzelnen Landgerichtsurteil.

Diese Rohdaten werden lokal gespeichert und anschließend in ein tabellarisches Format überführt. Hierzu werden die relevanten Felder aus den JSON-Strukturen extrahiert und in einer einheitlichen Datenstruktur zusammengeführt, beispielsweise in Form einer CSV-Datei. 
Der so erzeugte Datensatz bildet die Grundlage für die weitere statistische Auswertung und Analyse in den folgenden Abschnitten.

In [ ]:
# Hier Code einfügen

---

## 5. Datenaufbereitung

In diesem Abschnitt werden die im vorherigen Schritt erzeugten Analyseergebnisse weiterverarbeitet und für die nachgelagerte statistische Auswertung vorbereitet. Grundlage hierfür ist der aus den Batch-Ergebnissen abgeleitete strukturierte Datensatz, der die vom Sprachmodell extrahierten Informationen zu den einzelnen Urteilen enthält.

Ziel der Datenaufbereitung ist es, die extrahierten Merkmale in eine konsistente, auswertbare Form zu überführen, fehlende oder uneinheitliche Angaben zu behandeln und die Zielvariablen für die spätere Analyse eindeutig zu definieren.

In [ ]:
import pandas as pd
import re
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Setup: Spezialisiertes deutsches Sprachmodell laden
try:
    nlp = spacy.load("de_core_news_lg")
except Exception:
    print("Bitte installiere das spacy Modell: python -m spacy download de_core_news_lg")

# --- 2. JURISTISCHE TEXTVORVERARBEITUNG ---
def legal_preprocess(text):
    """
    Bereitet juristische Texte auf, indem Rauschen entfernt wird, 
    während rechtlich relevante Zahlen und Kontexte geschützt werden.
    """
    if not isinstance(text, str) or not text:
        return ""

    # NEU: START DES URTEILS FINDEN (Rauschschnitt Anfang) ---
    # Wir schneiden Webseiten-Menüs ("trending", "suche" etc.) weg
    start_keywords = ["tenor", "entscheidungsgründe", "tatbestand", "urteil", "beschluss", "endurteil"]
    text_lower_start = text.lower()
    
    # Finde die früheste Position eines der Keywords
    found_positions = [text_lower_start.find(kw) for kw in start_keywords if text_lower_start.find(kw) != -1]
    if found_positions:
        text = text[min(found_positions):]

    # NEU: ENDE DES URTEILS FINDEN (Rauschschnitt Ende) ---
    # Wir schneiden Impressum und Footer weg
    end_keywords = ["impressum", "nutzungsbedingungen", "nach oben", "datenschutz"]
    text_lower_end = text.lower()
    for ekw in end_keywords:
        e_pos = text_lower_end.find(ekw)
        if e_pos != -1:
            text = text[:e_pos]
            break

    # 1. Bereinigung von Rauschen (HTML-Tags, Sonderzeichen)
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Schutz von Zahlen & Paragraphen (Platzhalter statt Löschen)
    # Euro-Beträge schützen
    text = re.sub(r'\d{1,3}(?:\.\d{3})*(?:,\d+)?\s*(?:EUR|€|Euro)', ' PLATZHALTER_BETRAG ', text)
    # Paragraphen schützen
    text = re.sub(r'§+\s*\d+[a-z]?\s*(?:\w+)?', ' PLATZHALTER_PARAGRAPH ', text)
    # Jahreszahlen schützen
    text = re.sub(r'\b(19|20)\d{2}\b', ' PLATZHALTER_JAHR ', text)

    # 3. Kleinschreibung zur Reduktion der Varianz
    text = text.lower()

    # 4. Tokenisierung und Lemmatisierung mit SpaCy
    doc = nlp(text)
    
    # 5. Kontextsensitive Stoppwort-Entfernung
    # Wichtige juristische Negationen schützen
    protected_negations = {"nicht", "kein", "ohne", "gegen", "trotz"}
    custom_stop_words = nlp.Defaults.stop_words - protected_negations
    
    # Extraktion der Lemmata (Grundformen)
    tokens = [
        token.lemma_ for token in doc 
        if token.text not in custom_stop_words 
        and not token.is_punct 
        and not token.is_space
        and len(token.text) > 1 # Token mit Länge 1 entfernen
    ]
    
    return " ".join(tokens)

# --- 2.5 HILFSFUNKTIONEN: Simulation + echtes Batch lesen ---
def get_llm_text(r: dict) -> str:
    # Simulation (simulated_batch_output.jsonl)
    if "text" in r:
        return r["text"]
    # Echtes Batch (später)
    if "response" in r:
        return r["response"]["body"]["choices"][0]["message"]["content"]
    raise KeyError("Unbekanntes Ergebnisformat (kein 'text' und kein 'response').")

def parse_llm_json(text: str) -> dict:
    # Entfernt ```json ... ``` falls vorhanden
    text = re.sub(r"^```json\s*|\s*```$", "", text.strip(), flags=re.MULTILINE)
    # Falls außenrum Text steht: ersten JSON-Block extrahieren
    m = re.search(r"(\{.*\})", text, flags=re.DOTALL)
    if m:
        text = m.group(1)
    return json.loads(text)

# --- 3. MERGING DER DATEN (URTEILE + EXTRAKTIONEN) ---
def merge_and_finalize(judgment_file, batch_results_file):
    """
    Führt die ursprünglichen Urteilstexte mit den Gemini-Extraktionen zusammen.
    """
    # 1. Laden der aufbereiteten LG-Urteile
    df_judgments = pd.read_json(judgment_file, lines=True)
    df_judgments['case_id'] = df_judgments['custom_id'].str.replace('case_', '')

    # 2. Laden der Gemini-Batch-Ergebnisse
    with open(batch_results_file, 'r', encoding='utf-8') as f:
        results = [json.loads(line) for line in f]
    
    extracted_rows = []
    for r in results:
        try:
            case_id = r['custom_id'].replace('case_', '')
            llm_text = get_llm_text(r)
            content = parse_llm_json(llm_text)
            content['case_id'] = case_id
            extracted_rows.append(content)
        except Exception:
            continue
            
    df_extracted = pd.DataFrame(extracted_rows)

    # 3. Zusammenführung über case_id 
    df_final = pd.merge(df_judgments, df_extracted, on='case_id', how='inner')

    # 4. Textverarbeitung anwenden
    print("Starte Textvorverarbeitung...")
    df_final['cleaned_text'] = df_final['text'].apply(legal_preprocess)

    return df_final

# --- 4. MODELL-VORBEREITUNG (TF-IDF) ---
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),   # Bigramme erhalten Wortzusammenhänge
    max_features=1000,    # Reduktion der Komplexität
    min_df=5              # Seltene Begriffe ignorieren
)

In [ ]:
RESULTS_FILE = "simulated_batch_output.jsonl"

In [ ]:
df_final = merge_and_finalize(
    judgment_file="lg_judgments.jsonl",
    batch_results_file=RESULTS_FILE
)

print(df_final.shape)
df_final.head()


In [19]:
##Test der Datenvorverarbeitung auf einzelne Datei
import os
from pathlib import Path

# Pfad zu den Gerichtsurteilen
base_path = Path.cwd().parent 
pfad_zu_urteilen = base_path / "data" / "Gerichtsurteile_Openjur"

# Test mit einer Datei aus deiner Liste
test_datei_name = "2090187.txt" 
voller_pfad = pfad_zu_urteilen / test_datei_name

try:
    with open(voller_pfad, "r", encoding="utf-8") as f:
        original_text = f.read()
    
    # Schneller Test mit 2.000 Zeichen
    test_schnipsel = original_text[:2000]
    gereinigter_text = legal_preprocess(test_schnipsel)
    
    print(f"✅ Datei gefunden: {test_datei_name}")
    print("-" * 40)
    print(gereinigter_text[:500] + "...")

except FileNotFoundError:
    print(f"❌ Pfad falsch. Er sucht hier: {voller_pfad}")

✅ Datei gefunden: 2090187.txt
----------------------------------------
Endurteil 23.05. Platzhalter_jahr 21 658 16 Zitierung einfach lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 M. Fundstelle lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 Openjur Platzhalter_jahr 9516 Bibtex ris json lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 fundstell Openjur Platzhalter_jahr 9516 Rechtskraft Option Zitierung Version Tenor i. Klage abgewiesen.ii Kläger Kosten rechtsstreit tragen.iii Urteil gegen Sicherheitsleistung Höhe 1,1-fache vollstreckend betr...


Nachweis der Daten-Pipeline (Schritt 1 & 2)
Pfad-Validierung: Python-Skript versteht die Ordnerstruktur und kann die über 2.300 .txt-Dateien einlesen. 
Filter-Logik: Die Statistik zeigt, dass dein Code erfolgreich zwischen Landgerichten (LG) und Oberlandesgerichten (OLG) unterscheidet. 
Rauschschnitt (Slicing): Die Menüführung der Webseite (Trending, Spenden, Suche) ist abgeschnitten
Abstraktion durch Platzhalter: Dass PLATZHALTER_JAHR auftaucht, bestätigt, dass deine Regex-Logik funktioniert. Das Modell wird dadurch gezwungen, juristische Muster (wie „Klage abgewiesen“) zu lernen, statt sich an unwichtigen spezifischen Zahlen festzubeißen.

## 6. Analyse und Auswertung